In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten 
from tensorflow.keras.layers import BatchNormalization, Dropout, concatenate
from tensorflow.keras.models import Model

def build_parallel_cnn(input_shape, num_targets):
    # Входной слой
    main_input = Input(shape=input_shape, name='main_input')
    
    # Ветвь временной обработки
    temporal = Conv1D(64, 3, activation='relu', padding='same')(main_input)
    temporal = BatchNormalization()(temporal)
    temporal = MaxPooling1D(2, padding='same')(temporal)  # [None, 12, 64]
    
    # Ветвь сенсорных взаимодействий (1x1 свертки)
    sensor = Conv1D(64, 1, activation='relu', padding='same')(main_input)
    sensor = BatchNormalization()(sensor)
    sensor = MaxPooling1D(2, padding='same')(sensor)  # [None, 12, 64]
    
    # Совместимая конкатенация по размерности признаков
    merged = concatenate([temporal, sensor], axis=-1)  # [None, 12, 128]
    
    # Дополнительная обработка
    merged = Conv1D(128, 3, activation='relu', padding='same')(merged)
    merged = Flatten()(merged)
    merged = Dense(256, activation='relu')(merged)
    merged = Dropout(0.3)(merged)
    
    # Выходной слой
    output = Dense(num_targets, activation='linear')(merged)
    
    return Model(inputs=main_input, outputs=output)

# Проверка совместимости форм
if __name__ == "__main__":
    model = build_parallel_cnn((24, 13), 4)  # Пример параметров
    model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 24, 13)]     0           []                               
                                                                                                  
 conv1d_4 (Conv1D)              (None, 24, 64)       2560        ['main_input[0][0]']             
                                                                                                  
 conv1d_5 (Conv1D)              (None, 24, 64)       896         ['main_input[0][0]']             
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 24, 64)      256         ['conv1d_4[0][0]']               
 rmalization)                                                                                 

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import time

# 1. Загрузка данных
df = pd.read_excel('AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']], na_values=-200)
df.rename(columns={'Date_Time': 'timestamp'}, inplace=True)

# 2. Предобработка данных
# Обработка отсутствующих значений
df.interpolate(method='linear', limit_direction='both', inplace=True)

# Выбор целевой переменной (пример для CO(GT))
target = 'CO(GT)'
features = [col for col in df.columns if col not in ['timestamp', target]]

# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(df[features])
y = df[target].values.reshape(-1, 1)

# 3. Создание временных окон (подготовка данных для Conv1D)
def create_sequences(data, targets, window_size=24):
    X_seq, y_seq = [], []
    for i in range(len(data)-window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(targets[i+window_size])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X, y)

# 4. Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, shuffle=False)

# 5. Архитектура CNN
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', 
           input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(2),
    Conv1D(128, kernel_size=2, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)
])

# 6. Компиляция и обучение
model.compile(optimizer='adam', 
              loss='mse', 
              metrics=['mae', 'mse'])

start_time = time.time()
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=64,
                    validation_split=0.1,
                    verbose=1)
training_time = time.time() - start_time

# 7. Оценка модели
test_results = model.evaluate(X_test, y_test, verbose=0)
print(f'''
Результаты тестирования:
Время обучения: {training_time:.2f} сек
MAE: {test_results[1]:.4f}
MSE: {test_results[2]:.4f}
''')


C:\Users\User\AppData\Local\Temp\ipykernel_10612\4050362838.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_excel('AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']], na_values=-200)


Epoch 1/100
105/105 [==============================] - 5s 16ms/step - loss: 1.4840 - mae: 0.8906 - mse: 1.4840 - val_loss: 1.0353 - val_mae: 0.7370 - val_mse: 1.0353
Epoch 2/100
105/105 [==============================] - 1s 12ms/step - loss: 0.8087 - mae: 0.6567 - mse: 0.8087 - val_loss: 1.1587 - val_mae: 0.7418 - val_mse: 1.1587
Epoch 3/100
105/105 [==============================] - 1s 12ms/step - loss: 0.6612 - mae: 0.5910 - mse: 0.6612 - val_loss: 0.9890 - val_mae: 0.7219 - val_mse: 0.9890
Epoch 4/100
105/105 [==============================] - 1s 13ms/step - loss: 0.5942 - mae: 0.5622 - mse: 0.5942 - val_loss: 1.1638 - val_mae: 0.7464 - val_mse: 1.1638
Epoch 5/100
105/105 [==============================] - 1s 14ms/step - loss: 0.5715 - mae: 0.5524 - mse: 0.5715 - val_loss: 1.0825 - val_mae: 0.6958 - val_mse: 1.0825
Epoch 6/100
105/105 [==============================] - 1s 12ms/step - loss: 0.5143 - mae: 0.5204 - mse: 0.5143 - val_loss: 1.2214 - val_mae: 0.7663 - val_mse: 1.2214
Epoc